In [1]:
from sqlalchemy.orm import registry
from sqlalchemy import Column, Integer, String, Boolean

In [2]:
mapper_registry = registry()

In [3]:
@mapper_registry.mapped
class Partner:
    __tablename__ = 'partner'
    __table_args__ = {'extend_existing': True} 
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    is_active = Column(Boolean, nullable=False)
    
    def __repr__(self):
        return f'<Partner> {self.id} - {self.name}'

In [4]:
# the User class now has a Table object associated with it
Partner.__table__

Table('partner', MetaData(), Column('id', Integer(), table=<partner>, primary_key=True, nullable=False), Column('name', String(), table=<partner>, nullable=False), Column('is_active', Boolean(), table=<partner>, nullable=False), schema=None)

In [5]:
# The Mapper object mediates the relationship between User              
# and the "user" Table object.  This mapper object is generally behind  
# the scenes.
Partner.__mapper__

<Mapper at 0x7f9c3882f700; Partner>

In [6]:
boston_wine = Partner(name="Boston Wine Emporium", is_active=True)

In [7]:
boston_wine

<Partner> None - Boston Wine Emporium

In [8]:
boston_wine.is_active

True

In [9]:
boston_wine.name

'Boston Wine Emporium'

In [10]:
# Attributes which we didn't set, such as the "id", are displayed as  |
# None when we access them
repr(boston_wine.id)

'None'

In [11]:
# Using our registry, we can create a database schema for this class using
# a MetaData object that is part of the registry.
from sqlalchemy import create_engine

In [12]:
engine = create_engine("sqlite://")

In [13]:
with engine.begin() as connection:
    mapper_registry.metadata.create_all(connection)

In [14]:
# To persist and load User objects from the database, we           
#  use a Session object, illustrated here from a factory called     
#  sessionmaker.  The Session object makes use of a connection      
#  factory (i.e. an Engine) and will handle the job of connecting,  
#  committing, and releasing connections to this engine.

In [15]:
from sqlalchemy.orm import sessionmaker

In [16]:
Session = sessionmaker(bind=engine, future=True)

In [17]:
session = Session()

In [18]:
# new objects are placed into Session using add()

In [19]:
session.add(boston_wine)

In [20]:
# This did not yet modify the database, however the object is now known as      
# **pending**.  We can see the "pending" objects by looking at the session.new  
# attribute.
session.new

IdentitySet([<Partner> None - Boston Wine Emporium])

In [21]:
boston_wine

<Partner> None - Boston Wine Emporium

In [22]:
from sqlalchemy import select
select_statement = select(Partner).filter_by(name='Boston Wine Emporium')
result = session.execute(select_statement)

In [23]:
result

In [24]:
# We can get the data back from the result, in this case using the 
# .scalar() method which will return the first column of the first row.
also_winery = result.scalar()

In [25]:
also_winery

<Partner> 1 - Boston Wine Emporium

In [26]:
boston_wine.id

1

In [27]:
boston_wine is also_winery

True

In [28]:
session.identity_map.items()

[((__main__.Partner, (1,), None), <Partner> 1 - Boston Wine Emporium)]

In [29]:
session.add_all(
    [
        Partner(name='Arsenal Wine & Spirits', is_active=True),
        Partner(name='Sherry\'s Wine & Spirits', is_active=False)
    ])

In [30]:
boston_wine.is_active = False

In [31]:
# the Session can tell us which objects are dirty
session.dirty

IdentitySet([<Partner> 1 - Boston Wine Emporium])

In [32]:
# the Session can tell us which objects are pending
session.new

IdentitySet([<Partner> None - Arsenal Wine & Spirits, <Partner> None - Sherry's Wine & Spirits])

In [33]:
# The whole transaction is committed.  Commit always triggers
# a final flush of remaining changes.
session.commit()

In [34]:
# After a commit, theres no transaction.  The Session                
# *invalidates* all data, so that accessing them will automatically  
# start a *new* transaction and re-load from the database.  This is  
# our first example of the ORM *lazy loading* pattern.

boston_wine.is_active

False

In [35]:
# *** rolling back changes ***

# You try: 
    # Make another "dirty" change, and another "pending" change,
    # that we might change our minds about.

In [36]:
boston_wine.name = "Fake name for this Boston wine place"
session.add(Partner(name="I dont intend to keep this", is_active=False))

In [37]:
session.dirty

IdentitySet([<Partner> 1 - Fake name for this Boston wine place])

In [38]:
session.new

IdentitySet([<Partner> None - I dont intend to keep this])

In [39]:
result = session.execute(
    select(Partner).where(Partner.name.in_(['Fake name for this Boston wine place', 
                                  'I dont intend to keep this']))
    )


In [40]:
result.all()

[(<Partner> 1 - Fake name for this Boston wine place,),
 (<Partner> 4 - I dont intend to keep this,)]

In [41]:
session.rollback()

In [42]:
boston_wine.name

'Boston Wine Emporium'

In [43]:
result = session.execute(
    select(Partner).where(Partner.name.in_(['Boston Wine Emporium', 
                                  'I dont intend to keep this']))
    )

In [44]:
result.all()

[(<Partner> 1 - Boston Wine Emporium,)]

In [45]:
# Notes to myself

# Create engine first with the in-memory db
# Then create a session object
    # How do you see 'dirty' objects? 
    # How do you see 'pending' objects?
    # Check session by running: session.dirty and session.new
    # There are 5 different object states? Need to rewatch the Sqlalchemy 2021 video again
# Before you commit, you can session.rollback()

In [48]:
print(Partner.name == 'Boston Wine Emporium')

partner.name = :name_1


In [49]:
print(Partner.name == 'Boston')

partner.name = :name_1


In [90]:
query = (
    select(Partner).where(Partner.name == 'Arsenal Wine & Spirits').order_by(Partner.id)
)
result = session.execute(query)

In [91]:
print(result)

In [92]:
for row in result:
    print(row)

(<Partner> 2 - Arsenal Wine & Spirits,)


In [93]:
result

In [94]:
results = session.execute(query)

In [95]:
for partner in results.scalars():
    print(partner)

<Partner> 2 - Arsenal Wine & Spirits


In [110]:
query = select(Partner.name, Partner.is_active)
result = session.execute(query)

In [111]:
for row in result:
    print(f'{row.name}, {row.is_active}')

Boston Wine Emporium, False
Arsenal Wine & Spirits, True
Sherry's Wine & Spirits, False


In [113]:
query = select(Partner.id, Partner.name, Partner.is_active)
combo = session.execute(query)
for row in combo:
    print(f'{row.id}. {row.name}, {row.is_active}')

1. Boston Wine Emporium, False
2. Arsenal Wine & Spirits, True
3. Sherry's Wine & Spirits, False


In [130]:
for (name, ) in session.execute(
        select(Partner.name).filter_by(name='Boston Wine Emporium')
):
    print(name)

Boston Wine Emporium


In [132]:
from sqlalchemy import or_
for (partner, ) in session.execute(
    select(Partner)
    .where(or_(Partner.name == 'Sherry\'s Wine & Spirits', Partner.is_active == True))
):
    print(partner)

<Partner> 2 - Arsenal Wine & Spirits
<Partner> 3 - Sherry's Wine & Spirits
